In [1]:
__author__ = "Dong Qichen"
__license__ = "GPL"
__version__ = "3.0"

In [ ]:
%pip install ipywidgets ipympl --quiet

# Enable the custom widget manager, this is specific to the colab environment
from google.colab import output
output.enable_custom_widget_manager()

%matplotlib widget

# Display the scalar potential for a point charge, for which the speed (in units of c), beta, can be varied 
# by means of a slider 

import matplotlib.pyplot as plt
from matplotlib import cm, ticker
from ipywidgets import AppLayout, FloatSlider
plt.ioff()

import numpy as np


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


ModuleNotFoundError: No module named 'google'

In [ ]:
# Set fundamental constants relevant to electrodynamics.
e = 1  # Elementary charge.
c = 1  # Speed of light.
h = 2 * np.pi  # Planck's constant.
alpha = 1/137  # Fine-structure constant.
epsilon_0 = e**2 / (2 * alpha * h * c)  # Vacuum permittivity.

# Define a function 'calcA0' to calculate the vector potential (A0) at a given position (x_1, x_2) for a charged particle.
def calcA0(x_1, x_2, beta, q):
    # Calculate the distance from the source to the observation point (R) and the polar angle (theta).
    R = np.sqrt(x_1**2 + x_2**2)
    theta = np.arctan(x_2 / x_1)
    
    # Calculate the vector potential A0 using the formula for a moving charged particle.
    A0 = q / (4 * np.pi * epsilon_0 * c * R) / (1 - (beta * np.sin(theta))**2)**0.5
    
    return A0

# Define a function 'calcE1E2' to calculate the electric field components (E1, E2) and magnitude (abs_E) at a given position (x_1, x_2).
def calcE1E2(x_1, x_2, beta, q):
    # Calculate the distance from the source to the observation point (R) and the polar angle (theta).
    R = np.sqrt(x_1**2 + x_2**2)
    theta = np.arctan(x_2 / x_1)
    
    # Adjust the polar angle (theta) based on the particle's direction of motion.
    theta = theta + (x_1 < 0) * np.pi
    
    # Calculate the Lorentz factor (gamma) associated with the relativistic speed (beta).
    gamma = 1 / (1 - beta**2)**0.5
    
    # Calculate the magnitude of the electric field (abs_E) using the formula for a moving charged particle.
    abs_E = (q / (4 * np.pi * epsilon_0 * R**2)) * (1 / gamma**2) * (1 / (1 - beta**2 * np.sin(theta)**2))**1.5
    
    # Calculate the electric field components (E1 and E2) based on the polar angle (theta).
    E_1 = abs_E * np.cos(theta)
    E_2 = abs_E * np.sin(theta)
    
    return (E_1, E_2, abs_E)


In [ ]:
beta = 0
# Create a figure for visualizing electromagnetic fields with three subplots.
fig = plt.figure(figsize=(12, 4))

# Subplot 1: Vector Potential (A0)
ax1 = fig.add_subplot(131, autoscale_on=False, xlim=(-1, 1), ylim=(-1, 1), aspect='equal')
ax1.set_title('A0')

# Create a grid of x and y coordinates for A0 calculation.
x1_grid, x2_grid = np.meshgrid(np.linspace(-1, 1, 500), np.linspace(-1, 1, 500))

# Calculate A0 and create a pcolormesh plot for visualization.
mesh1 = ax1.pcolormesh(x1_grid, x2_grid, calcA0(x1_grid, x2_grid, beta=beta, q=1), cmap=cm.coolwarm, vmin=0, vmax=1.0)

# Add a text label to display information in this subplot.
text1 = ax1.text(0.02, 0.85, '', transform=ax1.transAxes, fontsize=25)

# Subplot 2: Electric Field Stream Plot
ax2 = fig.add_subplot(132, autoscale_on=False, xlim=(-1, 1), ylim=(-1, 1), aspect='equal')
ax2.set_title('E Field Stream Plot')

# Calculate the electric field components E_1, E_2, and E_abs.
E_1, E_2, E_abs = calcE1E2(x1_grid, x2_grid, beta=beta, q=1)

# Plot streamlines of the electric field using appropriate settings.
color = np.log(1 + E_abs)
lw    = color

# Create a circle of radius 0.1 centered at the origin. To determine the starting points for the streamlines,
step = np.pi/200
circle_theta = np.arange(0, 2*np.pi, step)
circle_r = 0.1
circle_x, circle_y = circle_r * np.cos(circle_theta), circle_r * np.sin(circle_theta)
e1_along_circle, e2_along_circle, eabs_along_circle = calcE1E2(circle_x, circle_y, beta=beta, q=1)
n_lines = 16

e_abs_integral_around_circle = np.cumsum(eabs_along_circle)
e_abs_integral_around_circle = e_abs_integral_around_circle / e_abs_integral_around_circle[-1]
starting_points_index  = []
for i in range(n_lines):
    starting_points_index.append(np.argmin(np.abs(e_abs_integral_around_circle - i/n_lines)))
starting_points_index = np.array(starting_points_index)
starting_points = np.array([circle_x[starting_points_index], circle_y[starting_points_index]]).T

# Create streamlines with arrowheads and colormap.
streams = ax2.streamplot(
    x1_grid, x2_grid, E_1, E_2,
    color=color,
    linewidth=lw,
    arrowstyle='->',
    arrowsize=1,
    broken_streamlines=False,
    start_points=starting_points,
)

# Subplot 3: Electric Field Contour Plot
ax3 = fig.add_subplot(133, autoscale_on=False, xlim=(-1, 1), ylim=(-1, 1), aspect='equal')
ax3.set_title('E Field Contour Plot')

# Create a contour plot to visualize the magnitude of the electric field.
contour = ax3.contour(x1_grid, x2_grid, E_abs, locator=plt.LogLocator())
fmt = ticker.LogFormatterMathtext()
fmt.create_dummy_axis()

# Label contour lines with logarithmic formatting.
ax3.clabel(contour, contour.levels, fmt=fmt)

<a list of 2 text.Text objects>

In [ ]:
# Create a FloatSlider widget to control the value of 'beta'.
slider = FloatSlider(
    orientation='horizontal',
    description='Beta:',
    value=0.0,
    step=0.00005,
    min=0.0,
    max=0.99999,
    readout=False,
)

# Set the width of the slider widget.
slider.layout.width = '90%'

# Define an update function that gets triggered when the slider value changes.
def update(beta_slider): 
    # Calculate the new value of 'beta' by taking the square root of the slider's value.
    beta_new = (beta_slider.new) ** 0.2
    
    # Calculate the vector potential (A0) with the updated 'beta'.
    A0s = calcA0(x1_grid, x2_grid, beta_new, 1)
    
    # Calculate the electric field components and magnitude with the updated 'beta'.
    E_1, E_2, E_abs = calcE1E2(x1_grid, x2_grid, beta=beta_new, q=1)
    
    # Update the pcolormesh plot for 'A0'.
    mesh1.set_array(A0s.ravel())
    
    # Update the text to display the current 'beta' value.
    text1.set_text('$\\beta$ = {:.7f}'.format(beta_new))
    
    # Clear and update the electric field stream plot.
    ax2.clear()
    ax2.set_title('E Field Stream Plot')
    color = np.log(1 + E_abs)
    lw    = 2 * color * (color < 1) + 2 * (color > 1)
    
    # Create a circle of radius 0.1 centered at the origin. To determine the starting points for the streamlines,
    step = np.pi/200
    circle_theta = np.arange(0, 2*np.pi, step)
    circle_r = 0.1
    circle_x, circle_y = circle_r * np.cos(circle_theta), circle_r * np.sin(circle_theta)
    e1_along_circle, e2_along_circle, eabs_along_circle = calcE1E2(circle_x, circle_y, beta=beta_new, q=1)
    n_lines = 16

    e_abs_integral_around_circle = np.cumsum(eabs_along_circle)
    e_abs_integral_around_circle = e_abs_integral_around_circle / e_abs_integral_around_circle[-1]
    starting_points_index  = []
    for i in range(n_lines):
        starting_points_index.append(np.argmin(np.abs(e_abs_integral_around_circle - i/n_lines)))
    starting_points_index = np.array(starting_points_index)
    starting_points = np.array([circle_x[starting_points_index], circle_y[starting_points_index]]).T
    
    streams = ax2.streamplot(
        x1_grid, x2_grid, E_1, E_2,
        color=color,
        linewidth=lw,
        arrowstyle='->',
        arrowsize=1,
        broken_streamlines=False,
        start_points=starting_points,
    )
    # Clear and update the electric field contour plot.
    ax3.clear()
    ax3.set_title('E Field Contour Plot')
    contour = ax3.contour(x1_grid, x2_grid, E_abs, locator=plt.LogLocator())
    fmt = ticker.LogFormatterMathtext()
    fmt.create_dummy_axis()
    levels = contour.levels[:4]
    ax3.clabel(contour, levels, fmt=fmt)
    
    # Redraw the figure canvas to reflect the updates.
    fig.canvas.draw()
    fig.canvas.flush_events()

# Observe changes in the slider's value and trigger the 'update' function accordingly.
slider.observe(update, names='value')

# Create an AppLayout with the canvas as the center and the slider as the footer.
AppLayout(
    center=fig.canvas,
    footer=slider,
)


AppLayout(children=(FloatSlider(value=0.0, description='Beta:', layout=Layout(grid_area='footer', width='90%')…

The scalar potential may be expressed by
$$\frac{V}{c}=A^0 = \frac{q}{4\pi \varepsilon_0 cR}\left(\frac{1}{1-\beta^2\sin^2\theta}\right)^{\frac{1}{2}},$$
where $\boldsymbol{R}$ is the vector from the current position  $x^1=\beta x^0, x^2=0,x^3=0$ of the point charge to the point at which the field is
evaluated. The magnitude of $R$ is given by
$$R^2  = \left[x^1-\beta x^0\right]^2+\left(x^{2}\right)^2+\left(x^{3}\right)^2.$$
 $\theta$ is the angle between $\boldsymbol{R}$ and the direction of motion.
 
 The magnitude of  $\boldsymbol{E}$ may be expressed as
$$E = \frac{q}{4\pi \varepsilon_0 R^2}\frac{1}{\gamma^2}\left(\frac{1}{1-\beta^2\sin^2\theta}\right)^{\frac{3}{2}}.$$